# Path Planning and Control
Here are the core codes and respective explanation of the path planning and control module.

We used a total of 4 classes to encapsulate the code. Among them:
- `UR3Robot` is mainly responsible for controlling the robotic arm’s movement, grasping, releasing, resetting, and obtaining the current coordinates/joint information.
- `BlockGripPose` mainly reads information about the grasping point from the tf node.
- `BlockColor` mainly retrieves the color of the current block from the ros topic.
- `BlockStatus` mainly records the current grasping status and returns the corresponding placement position for the current block.

Using these classes for encapsulation and calling them greatly simplified our debugging work.

For detailed information, please refer to the code comments.

`class UR3Robot`: motion controlling module relies on function `moveL()` and `moveJ()`.

In [ ]:
class Ur3Robot:
    def __init__(self):
        self.robot_ip = "192.168.1.101"
        self.robot_control =
rtde_control.RTDEControlInterface(self.robot_ip)
        self.robot_receive =
rtde_receive.RTDEReceiveInterface(self.robot_ip)
        self.serial = serial.Serial('/dev/ttyUSB0', 115200,
timeout=1, bytesize=8)
        self.height = [0.152, 0.24, 0.313]

    def move_to_pose(self, pose):
        '''
        笛卡尔空间，运动到指定的位姿
        '''
        self.robot_control.moveL(pose, speed=0.7, acceleration=0.3)
        # self.robot_control.startContactDetection()
        # print(dir(self.robot_control))
        print("Move to: ", pose)
    
    def move_to_joints(self, joints, speed1=0.7):
        '''
        关节空间，运动到指定的关节空间坐标
        '''
        self.robot_control.moveJ(joints, speed=speed1,
acceleration=0.3)
        print("Move to: ", joints)

    def get_pose(self):
        ''' 
        获取末端真实位姿
        '''
        return self.robot_receive.getActualTCPPose()

    def get_joints(self):
        ''' 
        获取真实关节角度
        '''
        return self.robot_receive.getActualQ()

    def grip(self):
        '''
        吸盘抓取命令
        '''
        self.serial.write(bytes.fromhex('01 06 00 02 00 01 E9 CA'))
        time.sleep(1)
        self.serial.read(8)
    
    def check_grip_status(self):
    '''
    检查抓取状态
    '''
    grip_status = self.serial.write(bytes.fromhex('01 03 00 41 00 01 D4 1E'))

    response = self.serial.read(7)
    time.sleep(0.1)
    print(response)
    if len(response) == 7:
        _, _, _, data, _ = struct.unpack('>BBBHh', response)
        print(data)
        if int(data) == 1:
            print("Grip!")
            return True
        else:
            print("Grip Failed!")
            return False

    def release(self):
        '''
        吸盘释放命令
        '''
        self.serial.write(bytes.fromhex('01 06 00 02 00 02 A9 CB'))
        self.serial.read(8)
        time.sleep(0.5)
        print("Release")

    def move_to_initial(self):
        '''
        运动到初始位置
        '''
        self.robot_control.moveJ(np.array([-0.09374410310854131,
        -1.2120822111712855, -1.431586097184046, -1.754958454762594,
        1.5917638540267944, 1.4068727493286133]
        ), speed=1, acceleration=0.7)

    def place(self, place_center, place_index):
        self.move_to_pose(place_center + [0.3451115686119684,
        -1.4124031162913535, 2.729045751694018, 0.01872167644650542])

        self.move_to_pose(place_center + [self.height[place_index],
        -1.4165484848374141, 2.7852779909410804, 0.02754228797325798])

        self.release()

        self.move_to_pose(place_center + [0.3451115686119684,
        -1.4124031162913535, 2.729045751694018, 0.01872167644650542])
        self.move_to_initial()

`class BlockGripPose`:

In [ ]:
# 返回抓取位姿
class BlockGripPose:
    def __init__(self):
        self.tf_buffer = tf2_ros.Buffer()
        self.listener = tf2_ros.TransformListener(self.tf_buffer)

    def get_grip_pose(self):
        '''
        获取抓取位姿
        '''
        rate = rospy.Rate(10.0)
        while not rospy.is_shutdown():
            try:
                transform_info = self.tf_buffer.lookup_transform("base", "grasping_link", rospy.Time(0))

                trans = transform_info.transform.translation
                trans = [trans.x, trans.y, trans.z]

                rot = transform_info.transform.rotation
                rot = [rot.x, rot.y, rot.z, rot.w]

                break
            except (tf2_ros.LookupException,
                    tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
                rate.sleep()
                continue

        rot_scipy = R.from_quat(rot)
        rot_matrix = rot_scipy.as_matrix()
        rot1_matrix = np.array([[1,0,0],[0,-1,0],[0, 0, -1]]) @ rot_matrix
        rot1 = R.from_matrix(rot1_matrix).as_rotvec()
        end_position = np.concatenate((trans, R.from_quat(rot).as_rotvec()), axis=None)

        return end_position

`class BlockColor`:

In [ ]:
# 读取、管理物块颜色，便于抓取策略
class BlockColor:
    def __init__(self):
        self.color = None
        self.sub = rospy.Subscriber("/block_color", String,
self.callback)
    def callback(self, msg):
        self.color = msg.data

`class BlockStatus`:

In [ ]:
# 与抓取、放置策略相关
class BlockStatus:
    def __init__(self, init_status="000"):
        # 从左到右的三位数字分别代表绿色、红色、紫色物块已经被抓取的数量
        self.grasped_blocks = {
            "green": int(init_status[0]),
            "red": int(init_status[1]),
            "purple": int(init_status[2])
        }

        # 不同颜色的放置位置
        self.placement_positions = {
            "green": [0.3292303515740091, 0.21691525614358914],
            "red": [0.044273501669568, -0.273954602584187],  # [0.18519487357734707, 0.22113136544140378],
            "purple": [0.06766491103939823, 0.23271185939803501]
        }

    def update_status(self, color):
        '''
        状态更新管理
        '''
        # 给定当前抓取颜色，返回放置位置，以及该颜色已抓取的数量(0,1,2)
        # 放置位置默认为第一个放置位置，如果index为1或2，则向上平移放置位置
        self.grasped_blocks[color] += 1
        return self.placement_positions[color], self.grasped_blocks[color] - 1

`main()` function:

In [ ]:
def main(block="000"):
    rospy.init_node("robot_move_ur3")
    robot = Ur3Robot()  # 运动, 抓取放置相关
    status = BlockStatus(block)  # 抓取状态相关
    color_detector = BlockColor()  # 抓取颜色
    grip_poser = BlockGripPose()  # 抓取位姿

    fail_times = 0

    while not rospy.is_shutdown():

        # 1. 返回初始位置 (无避挡)，等待抓取指令
        robot.move_to_initial()
        robot.release()

        # 2. 接收抓取指令，并读取颜色话题中的颜色
        grip_pose = grip_poser.get_grip_pose()
        box_color = color_detector.color

        # 3. 前往抓取
        robot.move_to_joints([0.08432325720787048,
        -1.5345171133624476, -1.4132922331349417, -1.720248047505514,
        1.5535264015197754, 0.8780661821363536]
        )

        ready_grip_pose = grip_pose.copy()
        ready_grip_pose[2] = 0.30086226203620025
        robot.move_to_pose(ready_grip_pose)
        grip_pose[2] = grip_pose[2] - fail_times * 0.01
        robot.move_to_pose(grip_pose)
        robot.grip()
        robot.move_to_joints([0.18046332897557568,
        -1.3607528845416468, -1.1536343733416956, -1.7376568953143519,
        1.60897696018219, 1.0921486616134644])

        grip_check = robot.check_grip_status()
        if not grip_check:
            fail_times += 1
            continue
        fail_times = 0

        # 4. 计算放置位姿
        place_center, place_index = status.update_status(box_color)

        # 5. 前往放置
        # robot.move_to_initial()
        robot.place(place_center, place_index)